In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
import numpy as np
import os
%matplotlib inline
import matplotlib.pyplot as plt
import glob
import pathlib

In [2]:
tf.__version__

'2.7.0'

## Data preprocessing

In [85]:
BATCH_SIZE = 32

In [86]:
image_input = sorted(tf.io.gfile.glob('./ISIC2018_Task1-2_Training_Input_x2/*.jpg'))
ground_truth = sorted(tf.io.gfile.glob('./ISIC2018_Task1_Training_GroundTruth_x2/*.png'))

In [87]:
image_input[:3]

['.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0000000.jpg',
 '.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0000001.jpg',
 '.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0000003.jpg']

In [88]:
ground_truth[:3]

['.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0000000_segmentation.png',
 '.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0000001_segmentation.png',
 '.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0000003_segmentation.png']

In [89]:
index = np.random.permutation(len(image_input))
image_input = np.array(image_input)[index]
ground_truth = np.array(ground_truth)[index]

In [90]:
image_input[:3]

array(['.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0012266.jpg',
       '.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0014825.jpg',
       '.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0006612.jpg'],
      dtype='<U53')

In [91]:
ground_truth[:3]

array(['.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0012266_segmentation.png',
       '.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0014825_segmentation.png',
       '.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0006612_segmentation.png'],
      dtype='<U70')

In [92]:
# Divide the dataset into training set, test set and val set with 6：2：2
length = len(image_input)
print(length)

image_input_val = image_input[:(int(length*0.2))]
print(len(image_input_val))
ground_truth_val = ground_truth[:(int(length*0.2))]
print(len(ground_truth_val))

2594
518
518


In [93]:
image_input_test = image_input[int(length*0.2):int(length*0.3)]
ground_truth_test = ground_truth[int(length*0.2):int(length*0.3)]

image_input_train = image_input[int(length*0.3):]
ground_truth_train = ground_truth[int(length*0.3):]

In [94]:
print(len(image_input_train))
print(len(ground_truth_train))

1816
1816


In [96]:
train_ds = tf.data.Dataset.from_tensor_slices((image_input_train, ground_truth_train))
val_ds = tf.data.Dataset.from_tensor_slices((image_input_val, ground_truth_val))
test_ds = tf.data.Dataset.from_tensor_slices((image_input_test, ground_truth_test))

In [95]:
# Image processing function
# Limited by the  memory of the GPU, choose the image size of 192, 192
def processing_jpg(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (192, 192))
    
    return image
  
def processing_png(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.resize(image, (192, 192))
    
    return image

In [29]:
# normalize function
def normal(image, ground):
    image = tf.cast(image, tf.float32) / 255.0
    ground = tf.cast(ground, tf.float32) / 255.0
    
    return image, ground

def load_image(image_path, ground_path):
    image = processing_jpg(image_path)
    ground = processing_png(ground_path)
    image, ground = normal(image, ground)

    return image, ground

In [97]:
train_ds = train_ds.map(load_image)
val_ds = val_ds.map(load_image)
test_ds = test_ds.map(load_image)

In [99]:
train_ds = train_ds.batch(BATCH_SIZE)
val_ds = val_ds.batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

## Build improved Unet model

In [100]:
def context_module(input_layer, filters):
    norm_1 = tfa.layers.InstanceNormalization()(input_layer)
    conv_1 = keras.layers.Conv2D(filters, (3, 3), padding = "same", activation = keras.layers.LeakyReLU(alpha = 0.01))(norm_1)
    drop_layer = keras.layers.Dropout(0.3)(conv_1)
    norm_2 = tfa.layers.InstanceNormalization()(drop_layer)
    conv_2 = keras.layers.Conv2D(filters, (3, 3), padding = "same", activation = keras.layers.LeakyReLU(alpha = 0.01))(norm_2)

    return conv_2

def upsampling_module(input_layer, filters):
    up_layer = keras.layers.UpSampling2D((2, 2))(input_layer)
    up_layer_2 = keras.layers.Conv2D(filters, (3, 3), padding = "same", activation = keras.layers.LeakyReLU(alpha = 0.01))(up_layer)
    norm_1 = tfa.layers.InstanceNormalization()(up_layer_2)
    
    return norm_1

def localization_module(input_layer, filters):
    conv_1 = keras.layers.Conv2D(filters, (3, 3), padding = "same", activation = keras.layers.LeakyReLU(alpha = 0.01))(input_layer)
    norm_1 = tfa.layers.InstanceNormalization()(conv_1)
    conv_2 = keras.layers.Conv2D(filters, (1, 1), padding = "same", activation = keras.layers.LeakyReLU(alpha = 0.01))(norm_1)
    norm_2 = tfa.layers.InstanceNormalization()(conv_2)

    return norm_2

In [101]:
def dice_coef(y_true, y_pred, smooth = 1.0):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)

    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    dice_coef_result = (2.0 * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

    return dice_coef_result

def dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [102]:
def improved_model():
    input_size = (192, 192, 3)
    
    input_layer = keras.layers.Input(shape=(input_size))
    # Down-sampling
    # layer 1
    # 3x3x3 convolution_1
    conv_1 = keras.layers.Conv2D(16, kernel_size=(3,3), padding="same")(input_layer)
    # context_module_1
    context_module_1 = context_module(conv_1, 16)
    # element-wise sum
    sum_1 = keras.layers.Add()([conv_1, context_module_1])
    
    # layer 2
    conv_2 = keras.layers.Conv2D(32, kernel_size=(3,3), padding="same", strides=(2,2))(sum_1)
    context_module_2 = context_module(conv_2, 32)
    sum_2 = keras.layers.Add()([conv_2, context_module_2])
    
    # layer 3
    conv_3 = keras.layers.Conv2D(64, kernel_size=(3,3), padding="same", strides=(2,2))(sum_2)
    context_module_3 = context_module(conv_3, 64)
    sum_3 = keras.layers.Add()([conv_3, context_module_3])
    
    # layer 4
    conv_4 = keras.layers.Conv2D(128, kernel_size=(3,3), padding="same", strides=(2,2))(sum_3)
    context_module_4 = context_module(conv_4, 128)
    sum_4 = keras.layers.Add()([conv_4, context_module_4])
    
    # layer 5
    conv_5 = keras.layers.Conv2D(256, kernel_size=(3,3), padding="same", strides=(2,2))(sum_4)
    context_module_5 = context_module(conv_5, 256)
    sum_5 = keras.layers.Add()([conv_5, context_module_5])
    
    # localization 
    up_sampling_1 = upsampling_module(sum_5, 128)
    concat_1 = keras.layers.Concatenate()([up_sampling_1, sum_4])
    
    
    # Up-sampling
    # layer 4
    localization_1 = localization_module(concat_1, 128)
    up_sampling_2 = upsampling_module(localization_1, 64)
    
    # layer 3
    concatenation_2 = keras.layers.Concatenate()([up_sampling_2, sum_3])
    localization_2 = localization_module(concatenation_2, 64)
    segmentation_1 = keras.layers.Conv2D(1, (1,1), padding="same", activation=keras.layers.LeakyReLU(alpha=0.01))(localization_2)
    up_sampling_3 = upsampling_module(localization_2, 32)
    
    # layer 2
    concatenation_3 = keras.layers.Concatenate()([up_sampling_3, sum_2])
    localization_3 = localization_module(concatenation_3, 32)
    segmentation_2 = keras.layers.Conv2D(1, (1,1), padding="same", activation=keras.layers.LeakyReLU(alpha=0.01))(localization_3)
    up_sampling_4 = upsampling_module(localization_3, 16)
    up_sampling_segament_1 = keras.layers.UpSampling2D(size=(2,2))(segmentation_1)
    
    sum_sum_1 = keras.layers.Add()([up_sampling_segament_1, segmentation_2])
    
    # layer 1
    concatenation_4 = keras.layers.Concatenate()([up_sampling_4, sum_1])
    conv_6 = keras.layers.Conv2D(32, kernel_size=(3,3), padding="same", activation=keras.layers.LeakyReLU(alpha=0.01))(concatenation_4)
    segmentation_3 = keras.layers.Conv2D(1, (1,1), padding="same", activation=keras.layers.LeakyReLU(alpha=0.01))(conv_6)
    
    up_sampling_segament_2 = keras.layers.UpSampling2D(size=(2,2))(sum_sum_1)
    sum_segmentation = keras.layers.Add()([up_sampling_segament_2, segmentation_3])
    output_layer = keras.layers.Conv2D(filters=1, kernel_size=(1,1), activation='sigmoid', padding='same')(sum_segmentation)
    
    model = keras.Model(inputs=input_layer, outputs=output_layer)
    
    return model
    
    
    

In [103]:
model = improved_model()
model.compile(optimizer="adam", loss=dice_coef_loss, metrics=[dice_coef])

In [106]:
# model.summary()

In [105]:
history = model.fit(train_ds, validation_data = val_ds, epochs = 20)

Epoch 1/20
57/57 [==============================] - 14s 184ms/step - loss: 0.5289 - dice_coef: 0.4715 - val_loss: 0.5002 - val_dice_coef: 0.5043
Epoch 2/20
57/57 [==============================] - 10s 176ms/step - loss: 0.3805 - dice_coef: 0.6205 - val_loss: 0.2287 - val_dice_coef: 0.7753
Epoch 3/20
57/57 [==============================] - 10s 174ms/step - loss: 0.1914 - dice_coef: 0.8090 - val_loss: 0.1675 - val_dice_coef: 0.8350
Epoch 4/20
57/57 [==============================] - 10s 173ms/step - loss: 0.1577 - dice_coef: 0.8426 - val_loss: 0.1591 - val_dice_coef: 0.8435
Epoch 5/20
57/57 [==============================] - 10s 176ms/step - loss: 0.1420 - dice_coef: 0.8583 - val_loss: 0.1530 - val_dice_coef: 0.8473
Epoch 6/20
57/57 [==============================] - 10s 174ms/step - loss: 0.1349 - dice_coef: 0.8653 - val_loss: 0.1311 - val_dice_coef: 0.8707
Epoch 7/20
57/57 [==============================] - 10s 174ms/step - loss: 0.1252 - dice_coef: 0.8751 - val_loss: 0.1274 - val_dic